# Redes Sociais - APS 2

### Alunos: Arthur Barreto, Enricco Gemha e Felipe Catapano


Uma rede de artistas do Spotify que se conectam através de participar em determinada música, que foi topo das paradas em alguma semana no mundo, durante o período de 28/09/2013 a 09/10/2022. Os vértices representam artistas do Spotify e uma aresta não-direcionada indica uma música feita em parceria por dois artistas.

_Nota: os grafos são não-dirigidos, conforme informado no agregador de databases [Kaggle](https://www.kaggle.com/datasets/jfreyberg/spotify-artist-feature-collaboration-network), do qual foi extraído a base utilizada aqui._

## Pré-requisitos

In [ ]:
import graph_tool_extras as gte
import netpixi
from graph_tool import draw
import distribution as dst

In [ ]:
PATH = 'edges.csv'

## Análise dos dados importados

O arquivo relevante para nossa análise é `edges.csv`, que representa uma lista de valores separados por _vírgulas_, com uma aresta por linha. O significado de cada coluna do arquivo é:

- Primeira coluna: ID do nó de participação em música
- Segunda coluna: ID do nó de participação em música

Agora, devemos inspecionar os valores armazenados em `edges.csv`:

In [ ]:
with open(PATH) as file:

    # cria index de contagem para o loop abaixo.
    i = 0
    
    # ignora o cabeçalho.
    next(file)

    # Para não sobrecarregar este notebook
    # vamos espiar somente as 5 primeiras linhas.
    for line in file:

        # Transforma a linha em uma lista de partes,
        # considerando a vírgula como separador.
        parts = line.split(',')

        # Imprime o nó A que referencia e o nó B que é referenciado.
        print(parts[0], parts[1])

        # Para não sobrecarregar este notebook, vamos usar um contador
        # e um break para imprimir apenas as cinco primeiras linhas.
        i += 1
        if i == 5:
            break

## Criação do grafo

Utilizaremos a biblioteca [graph-tool](https://graph-tool.skewed.de/) somente para criação e visualização básica dos grafos, sem suporte de nenhum método ou função que não seja essencial.

In [ ]:
g = gte.Graph(directed=False) # pois o grafo não é direcionado, como informado acima.

Antes, vamos definir duas funções auxiliares para facilitar a adição de novos nós e arestas, respectivamente.

In [ ]:
def get_or_add_vertex(g, id):
    u = g.vertex_by_id(id)
    if u is None:
        u = g.add_vertex_by_id(id)
    return u

def get_or_add_edge(g, id1, id2):
    e = g.edge_by_ids(id1, id2)
    if e is None:
        e = g.add_edge_by_ids(id1, id2)
    return e

Depois de criar o novo grafo, vamos armazenar os valores de `edges.csv` nele, o transformando em uma rede.

In [ ]:
with open(PATH) as file:

    # Cria index de contagem de linhas lidas.
    i = 1
    
    # Ignora o cabeçalho.
    next(file)

    # Itera linha a linha do arquivo `out.linux`
    for line in file:

        # Transforma a linha em uma lista de partes,
        # considerando a vírgula como separador.
        parts = line.split(',')

        # Define os IDs de origem e destino.
        id1 = str(parts[0])
        id2 = str(parts[1])
        
        # Adiciona os vértices.
        get_or_add_vertex(g, id1)
        get_or_add_vertex(g, id2)

        # Adiciona a aresta correspondente a esta linha.
        get_or_add_edge(g, id1, id2)
        
        # Incrementa o contador de linhas lidas.
        i += 1

# Imprime a quantidade de linhas lidas.
print(f'Foram lidas {i} linhas.') 

A seguir, devemos chamar `draw.sfdp_layout`, passando a rede, para rodar um algoritmo de posicionamento chamado SFDP [[1](#sfdp)].

Esse algoritmo usa uma ideia conhecida como [force-directed graph drawing](https://en.wikipedia.org/wiki/Force-directed_graph_drawing) para posicionar os vértices de forma a evidenciar agrupamentos.

In [ ]:
layout = draw.sfdp_layout(g)

In [ ]:
gte.move(g, layout)

## Armazenamento da rede

Para garantir a segurança da informação processada, devemos guardá-la em um arquivo na mesma pasta deste notebook.

In [ ]:
g = gte.clean(g)
gte.save(g, 'spotify.net.gz')

## Visualização da rede

O próximo passo é a renderização da rede.

In [ ]:
r = netpixi.render('spotify.net.gz', infinite=True)

Por fim, devemos ajustar a visualização da renderização.

In [ ]:
r.vertex_default(size=4, bwidth=1)

In [ ]:
r.edge_default(width=1)

## Estatísticas

In [ ]:
print(f"Número de vértices: {len(g.get_vertices())}")
print(f"Número de arestas: {len(g.get_edges())}")

In [ ]:
print(f"Densidade da rede: {g.density()}")
print(f"Transitividade da rede: {g.transitivity()}")

In [ ]:
degrees = g.get_total_degrees()

In [ ]:
degrees.describe()

In [ ]:
degrees.hist(bins=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]);

In [ ]:
dst.not_normal(degrees)

In [ ]:
dst.more_powerlaw_than_lognormal(degrees)

In [ ]:
dst.more_powerlaw_than_exponential(degrees)

In [ ]:
distances = g.get_distances()

In [ ]:
distances.describe()

In [ ]:
distances.hist();